In [2]:
import pandas as pd
import os
import numpy  as np

In [3]:


# 查看文件中的所有键（可选）
with pd.HDFStore(output_file, mode='r') as store:
    keys = store.keys()
    print(f"文件中的所有键: {keys}")

# 抽取一个键的数据并转换为 pandas DataFrame
# 假设你想抽取键 '/RB' 的数据
key_to_extract = '/RB'  # 替换为你想提取的键
df = pd.read_hdf(output_file, key=key_to_extract)

# 打印 DataFrame 的前几行
print(f"\n提取的键: {key_to_extract}")
print(df.head())

# 打印 DataFrame 的基本信息
print("\nDataFrame 基本信息:")
print(df.info())

# 打印 DataFrame 的统计信息
print("\nDataFrame 统计信息:")
print(df.describe())

文件中的所有键: ['/A', '/AG', '/AL', '/AO', '/AP', '/AU', '/B', '/BB', '/BC', '/BR', '/BU', '/C', '/CF', '/CJ', '/CS', '/CU', '/CY', '/EB', '/EC', '/EG', '/FB', '/FG', '/FU', '/HC', '/I', '/IC', '/IF', '/IH', '/IM', '/J', '/JD', '/JM', '/JR', '/L', '/LC', '/LG', '/LH', '/LR', '/LU', '/M', '/MA', '/NI', '/NR', '/OI', '/P', '/PB', '/PF', '/PG', '/PK', '/PM', '/PP', '/PR', '/PS', '/PX', '/RB', '/RI', '/RM', '/RR', '/RS', '/RU', '/SA', '/SC', '/SF', '/SH', '/SI', '/SM', '/SN', '/SP', '/SR', '/SS', '/T', '/TA', '/TF', '/TL', '/TS', '/UR', '/V', '/WH', '/WR', '/Y', '/ZC', '/ZN']

提取的键: /RB
                             dominant_id    open   close    high     low  \
underlying_symbol date                                                     
RB                2017-01-03      RB1705  2889.0  2851.0  2998.0  2839.0   
                  2017-01-04      RB1705  2860.0  2930.0  2957.0  2827.0   
                  2017-01-05      RB1705  2930.0  2941.0  2998.0  2913.0   
                  2017-01-06      RB

In [4]:
df.columns

Index(['dominant_id', 'open', 'close', 'high', 'low', 'total_turnover',
       'volume', 'prev_close', 'settlement', 'prev_settlement',
       'open_interest', 'limit_up', 'limit_down', 'day_session_open'],
      dtype='object')

In [4]:
# 抽取数据代码

# HDF5 文件路径
output_file = 'D:\\FutureData\\ricequant\\1d_2017to2024_noadjust.h5'
# 读取铁矿石和螺纹钢的数据
df_I = pd.read_hdf(output_file, key='/I').reset_index().set_index('date')
df_RB = pd.read_hdf(output_file, key='/RB').reset_index().set_index('date')

In [7]:

# 选择收盘价
df_I = df_I[['close']]
df_RB = df_RB[['close']]

In [8]:
# 合并数据
merged_df = pd.concat([df_I.rename(columns={'close': 'I_close'}), df_RB.rename(columns={'close': 'RB_close'})], axis=1)

merged_df

,I_close,RB_close
date,,
2017-01-03,534.0,2851.0
2017-01-04,542.0,2930.0
2017-01-05,550.5,2941.0
2017-01-06,538.5,2912.0
2017-01-09,571.5,3048.0
...,...,...
2024-12-25,776.0,3300.0
2024-12-26,776.5,3305.0
2024-12-27,759.5,3268.0


In [9]:
merged_df['spread'] = merged_df['I_close'] - merged_df['RB_close']
merged_df

,I_close,RB_close,spread
date,,,
2017-01-03,534.0,2851.0,-2317.0
2017-01-04,542.0,2930.0,-2388.0
2017-01-05,550.5,2941.0,-2390.5
2017-01-06,538.5,2912.0,-2373.5
2017-01-09,571.5,3048.0,-2476.5
...,...,...,...
2024-12-25,776.0,3300.0,-2524.0
2024-12-26,776.5,3305.0,-2528.5
2024-12-27,759.5,3268.0,-2508.5


In [12]:
merged_df

,I_close,RB_close,spread
date,,,
2017-01-03,534.0,2851.0,-2317.0
2017-01-04,542.0,2930.0,-2388.0
2017-01-05,550.5,2941.0,-2390.5
2017-01-06,538.5,2912.0,-2373.5
2017-01-09,571.5,3048.0,-2476.5
...,...,...,...
2024-12-25,776.0,3300.0,-2524.0
2024-12-26,776.5,3305.0,-2528.5
2024-12-27,759.5,3268.0,-2508.5


In [34]:
import argparse
import datetime
import random

import backtrader as bt

In [28]:
spread_data = merged_df.copy()
# spread_data包含'date'和'spread'列，其中'date'是DatetimeIndex
spread_data.set_index('date', inplace=True)
spread_data['open'] = spread_data['spread']
spread_data['high'] = spread_data['spread']
spread_data['low'] = spread_data['spread']
spread_data['close'] = spread_data['spread']


,underlying_symbol,date,close
0,RB,2017-01-03,2851.0
1,RB,2017-01-04,2930.0
2,RB,2017-01-05,2941.0
3,RB,2017-01-06,2912.0
4,RB,2017-01-09,3048.0
...,...,...,...
1938,RB,2024-12-25,3300.0
1939,RB,2024-12-26,3305.0
1940,RB,2024-12-27,3268.0
1941,RB,2024-12-30,3317.0


In [40]:
import backtrader as bt
import pandas as pd
spread_data = merged_df.copy()
# 定义策略类
class SimpleMASpreadStrategy(bt.Strategy):
    params = (
        ('short_period', 10),
        ('long_period', 50),
    )

    def __init__(self):
        self.short_ma = bt.indicators.MovingAverageSimple(self.data, period=self.p.short_period)
        self.long_ma = bt.indicators.MovingAverageSimple(self.data, period=self.p.long_period)
        self.crossover = bt.indicators.CrossOver(self.short_ma, self.long_ma)
        self.buy_signal = self.crossover > 0
        self.sell_signal = self.crossover < 0
        self.obs = bt.observers.BuySell()
        self.obs_value = bt.observers.Value()

    def next(self):
        # if not self.position:
        #     if self.buy_signal:
        #         self.buy()
        # else:
        #     if self.sell_signal:
        #         self.sell()
        pass
# 加载数据
# 假设 spread_data 已经加载并以 'date' 为索引
# spread_data = pd.read_csv('your_data.csv', parse_dates=['date'], index_col='date')
# spread_data['spread'] = spread_data['I_close'] - spread_data['RB_close']
# spread_data.dropna(inplace=True)

# 确保 'spread_data' 包含 'date' 作为索引，并且有 'spread' 这一列
data = bt.feeds.PandasData(dataname=spread_data, open='spread', high='spread', low='spread', close='spread')

# 创建Cerebro引擎
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(SimpleMASpreadStrategy)
cerebro.broker.set_cash(100000)
cerebro.addsizer(bt.sizers.FixedSize, stake=1)

# 添加绘图观察者
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)

# 运行回测
cerebro.run()

# 绘图
cerebro.plot(style='candle', volume=False)

AttributeError: 'Lines_LineSeries_LineIterator_DataAccessor_Observe' object has no attribute '_fundmode'

In [42]:
cerebro = bt.Cerebro(stdstats=False)

# Add a strategy
cerebro.addstrategy(bt.Strategy)

# Get a pandas dataframe
# datapath = ('../../datas/2006-day-001.txt')?
#
# # Simulate the header row isn't there if noheaders requested
# skiprows = 1 if args.noheaders else 0
# header = None if args.noheaders else 0

# dataframe = pandas.read_csv(
#     datapath,
#     skiprows=skiprows,
#     header=header,
#     # parse_dates=[0],
#     parse_dates=True,
#     index_col=0,
# )

# if not args.noprint:
#     print('--------------------------------------------------')
#     print(dataframe)
#     print('--------------------------------------------------')

# Pass it to the backtrader datafeed and add it to the cerebro
data = bt.feeds.PandasData(dataname=spread_data,
                           # datetime='Date',
                           nocase=True,
                           )

cerebro.adddata(data)

# Run over everything
cerebro.run()

# Plot the result
cerebro.plot(style='bar')


ValueError: Axis limits cannot be NaN or Inf

In [54]:
# from __future__ import (absolute_import, division, print_function,
#                         unicode_literals)
#
# import backtrader as bt


class St(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data)


data =bt.feeds.PandasData(dataname=spread_data.reset_index(),
                           datetime='date',
                           nocase=True,
                           )

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(St)
cerebro.run()
cerebro.plot()

ValueError: Axis limits cannot be NaN or Inf

In [52]:
df_I = pd.read_hdf(output_file, key='/I').reset_index()

In [5]:
# 读取数据
output_file = 'D://y5//RB_daily.h5'
df_RB = pd.read_hdf(output_file, key='data')


In [10]:
r = df_RB.close.pct_change()   #品种策略的日度收益率 
r.head(20)
# (1 + r).cumprod()


date
20090330         NaN
20090331    0.001411
20090401   -0.000564
20090402   -0.020863
20090403   -0.007198
20090407    0.012471
20090408   -0.008021
20090409    0.001444
20090410    0.004614
20090413   -0.004305
20090414    0.026232
20090415    0.000562
20090416    0.009545
20090417    0.006674
20090420    0.000829
20090421   -0.027325
20090422    0.000568
20090423   -0.003687
20090424   -0.003416
20090427   -0.000571
Name: close, dtype: float64

In [8]:
(1 + r).cumprod()

date
20090330         NaN
20090331    1.001411
20090401    1.000847
20090402    0.979966
20090403    0.972912
              ...   
20241231    1.110308
20250102    1.109301
20250103    1.097893
20250106    1.091182
20250107    1.086820
Name: close, Length: 3836, dtype: float64

In [9]:
df_RB

,date,InstrumentID,open,high,low,close,volume,openint,create_time,update_time
date,,,,,,,,,,
20090330,20090330,RB,996.910980,1005.335580,990.732940,995.226060,145168.0,48380.0,2019-06-06 11:44:23.773,2019-06-06 11:44:23.773
20090331,20090331,RB,993.541140,1001.404100,991.575400,996.630160,70592.0,44714.0,2019-06-06 11:44:23.787,2019-06-06 11:44:23.787
20090401,20090401,RB,999.719180,1000.000000,994.945240,996.068520,28100.0,42076.0,2019-06-06 11:44:23.797,2019-06-06 11:44:23.797
20090402,20090402,RB,995.506880,996.349340,970.513901,975.287840,235446.0,68888.0,2019-06-06 11:44:23.810,2019-06-06 11:44:23.810
20090403,20090403,RB,977.534400,977.534400,964.897501,968.267341,152376.0,75038.0,2019-06-06 11:44:23.823,2019-06-06 11:44:23.823
...,...,...,...,...,...,...,...,...,...,...
20241231,20241231,RB,1110.684251,1111.018191,1102.669692,1105.007272,1115370.0,1561545.0,2024-12-31 16:47:22.057,2024-12-31 16:47:22.057
20250102,20250102,RB,1103.671512,1113.689710,1096.658773,1104.005452,1373928.0,1551112.0,2025-01-02 16:41:51.897,2025-01-02 16:41:51.897
20250103,20250103,RB,1103.671512,1105.007272,1088.644215,1092.651494,1560003.0,1675464.0,2025-01-03 16:56:10.320,2025-01-03 16:56:10.320


In [11]:
r = df_RB.close.pct_change()  # 品种策略的日度收益率

# 修改第二行（第二天）的收益率为开盘价和收盘价之差
r.iloc[1] = (df_RB.close.iloc[1] - df_RB.open.iloc[1]) / df_RB.open.iloc[1]

r.head(20)
(1 + r).cumprod()


date
20090330         NaN
20090331    1.003109
20090401    1.002544
20090402    0.981628
20090403    0.974562
              ...   
20241231    1.112191
20250102    1.111182
20250103    1.099755
20250106    1.093032
20250107    1.088663
Name: close, Length: 3836, dtype: float64